In [313]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import requests
import re
import pandas as pd

## Scrape NASA Mars news titles and paragraph

In [135]:
# open browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = "https://mars.nasa.gov/news/"
browser.visit(url)


In [141]:
# create soup object
html = browser.html
soup = bs(html, "html5lib")


In [171]:
# scrape data and save into list
news_title =[]
news_p = []

list_of_titles = soup.find("ul","item_list").find_all("div","content_title")
list_of_p = soup.find("ul","item_list").find_all("div","article_teaser_body")

for title, paragraph in zip(list_of_titles, list_of_p):
    try:
        news_title.append(title.text)
        news_p.append(paragraph.text)
    except:
        print("----error----")

        


## Scrape NASA Mars project Images

In [193]:
# open browser and scrape images
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url)

In [194]:
# scrape soup object
html = browser.html
soup = bs(html, "html5lib")


In [227]:
# scrape image url and store into a list
image_list = soup.find("section","grid_gallery").find_all("li","slide")
part_url = "https://www.jpl.nasa.gov/"
image_url = []

for image in image_list:
    try:
        url_img = image.find('img', {'src':re.compile('.jpg')})["src"]
        full_image_url = part_url+url_img
        image_url.append(full_image_url)
    except:
        print("----error----")


## Scrape Mars weather twitter

In [298]:
# open browser and scrape twitter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [299]:
# scrape soup object
html = browser.html
soup = bs(html, "html5lib")

In [310]:
twitters = soup.find_all("div","css-1dbjc4n")

In [311]:
mars_weather = []
for index, twit in enumerate(twitters):
    
    try:
        if twit.span:
            tweet = twit.find("span","css-901oao").text
            if tweet[0:11] == "InSight sol" and (tweet not in mars_weather) :
                mars_weather.append(tweet)
    except:
        print("=========")

        
        

## Scrape Mars facts

In [339]:
# use pandas to scrape table
url = "https://space-facts.com/mars/"
fact_table = pd.read_html(url)[0]
fact_table = fact_table.rename(columns = {0:"Metric",1:"Value"}).set_index("Metric")
fact_table_html = fact_table.to_html()


## Scrape Mars Hemispheres

In [385]:
# open browser and scrape images and titles
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

hemisphere_urls = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_urls)

In [386]:
# scrape soup object
html = browser.html
soup = bs(html, "html5lib")

In [387]:
# get list of titles and then click into each url to get full rev images
base_url = "https://astrogeology.usgs.gov"
titles = soup.find_all("div","description")
titles[0].h3.text

'Cerberus Hemisphere Enhanced'

In [388]:
title_list = []
image_url = []
for title in titles:
    try:
        title_list.append(title.h3.text)
        my_link = base_url+title.a["href"]
        
        # click the title link
        browser.links.find_by_partial_text(title.h3.text).click()
        
        # scrape image url
        new_html = browser.html
        new_soup = bs(new_html, "html5lib")
        # try to get image url
        try:
            img_url = new_soup.find("div","downloads").find("li").a["href"]
            image_url.append(img_url)
        except:
            print("----Image error----")
        
        # go back 
        browser.back()
        
        
    except:
        print("----error----")
    

In [408]:
# save title and image url into list of dict
for i in range(len(title_list)):
    dict_list = {"title":title_list[i],"img_url":image_url[i]}
    hemisphere_image_urls.append(dict_list)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]